Vehicles

In [ ]:
import pandas as pd

# Cargar el archivo CSV con los datos de costos operacionales de vehículos
costo_operacional_vehiculos = pd.read_csv('Data/costo_operacional_vehiculos_clean.csv')

# Crear el dataset Vehicles a partir del archivo CSV
vehicles_dataset = costo_operacional_vehiculos[['Manuf', 'Model', 'Fuel_Type', 'Fuel_Cost', 'Electric_Cost', 'Total_Cost', 'Noise_Level']].copy()

# Renombrar las columnas para que coincidan con la estructura deseada
vehicles_dataset.rename(columns={
    'Manuf': 'manuf',
    'Model': 'model',
    'Fuel_Type': 'fuel_type',
    'Fuel_Cost': 'fuel_cost',
    'Electric_Cost': 'electric_cost',
    'Total_Cost': 'total_cost',
    'Noise_Level': 'noise_level'
}, inplace=True)

# El campo 'model' será la clave primaria (PK)
# Asegúrate de que 'model' sea único
vehicles_dataset = vehicles_dataset.drop_duplicates(subset='model')

# Guardar el nuevo dataset
vehicles_dataset.to_parquet('etl_minio/vehicles.parquet', index=False)

print("El dataset 'Vehicles' ha sido creado y guardado en 'vehicles.parquet'.")


Trips (Yellow, Green, FHV)

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Cargar los datos
yellow_tripdata = pd.read_parquet('Data/yellow_tripdata.parquet')
green_tripdata = pd.read_parquet('Data/green_tripdata.parquet')
fhv_tripdata = pd.read_parquet('Data/fhv_tripdata.parquet')

# Agregar la columna 'tipo' a cada DataFrame
yellow_tripdata['tipo'] = 'Yellow'
green_tripdata['tipo'] = 'Green'
fhv_tripdata['tipo'] = 'FHV'

# Crear DataFrame combinado
combined_tripdata = pd.concat([yellow_tripdata, green_tripdata, fhv_tripdata], ignore_index=True)

# Crear un DataFrame que combine todas las ubicaciones únicas
unique_locations = pd.concat([
   combined_tripdata[['PULocationID']].rename(columns={'PULocationID': 'LocationID'}),
    combined_tripdata[['DOLocationID']].rename(columns={'DOLocationID': 'LocationID'})
]).drop_duplicates()
 
# Crear un geo_join_id único para cada LocationID
unique_locations['geo_join_id'] = range(1, len(unique_locations) + 1)

# Crear un mapping de LocationID a geo_join_id
location_to_geo_id = unique_locations.set_index('LocationID')['geo_join_id'].to_dict()

# Asignar geo_join_id a los datos de viajes
combined_tripdata['PULocation_geo_join_id'] = combined_tripdata['PULocationID'].map(location_to_geo_id)
combined_tripdata['DOLocation_geo_join_id'] = combined_tripdata['DOLocationID'].map(location_to_geo_id)


# Seleccionar y renombrar las columnas para el nuevo dataset
tripdata_dataset = combined_tripdata[['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'total_amount', 'PULocation_geo_join_id', 'DOLocation_geo_join_id', 'payment_type', 'congestion_surcharge', 'tipo']].copy()

# Limpiar las columnas para la clave foránea
tripdata_dataset.rename(columns={
    'VendorID': 'vendor_id',
    'pickup_datetime': 'pickup_datetime',
    'dropoff_datetime': 'dropoff_datetime',
    'passenger_count': 'passenger_count',
    'trip_distance': 'trip_distance',
    'total_amount': 'total_amount',
    'PULocation_geo_join_id': 'PULocationID',
    'DOLocation_geo_join_id': 'DOLocationID',
    'payment_type': 'payment_type',
    'congestion_surcharge': 'congestion_surcharge'
}, inplace=True)

# Seleccionar las columnas necesarias
tripdata_dataset =  tripdata_dataset[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'total_amount', 'PULocationID', 'DOLocationID', 'payment_type', 'congestion_surcharge', 'tipo']]

# Guardar el nuevo dataset

# Especificar la compresión (snappy, gzip, brotli, etc.)
compression_type = 'snappy'  # Cambiar por 'gzip', 'brotli', etc.

# Convertir DataFrame a un Table de PyArrow
table = pa.Table.from_pandas(tripdata_dataset)

# Guardar como archivo Parquet con compresión
pq.write_table(table, 'etl_minio/trips.parquet', compression=compression_type)

unique_locations.to_parquet('Data\datos_geograficos_limpios.parquet', index=False)

print("El dataset 'Trips' ha sido creado y guardado en 'trips_dataset.csv'.")

Geographic Data

In [ ]:
import pandas as pd

# Cargar los datos geográficos desde el archivo parquet
geographic_data = pd.read_parquet('Data/datos_geograficos_limpios.parquet')
air_quality = pd.read_parquet('Data/air_quality_cleaned.parquet')


# Asegurarse de que 'geo_join_id' sea único (clave primaria)
geographic_data.drop_duplicates(subset=['geo_join_id'], inplace=True)

# Guardar el nuevo dataset
geographic_data.to_parquet('etl_minio/geographic_data.parquet', index=False)

print("El dataset de datos geográficos ha sido creado y guardado en 'geographic_data.parquet'.")


 Air Quality Measurement

In [ ]:
import pandas as pd

# Cargar el archivo parquet con datos de calidad del aire
air_quality = pd.read_parquet('Data/air_quality_cleaned.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
air_quality_dataset = air_quality[['Name', 'Measure', 'Geo Place Name', 'Time Period', 'Data Value', 'Year']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
air_quality_dataset.rename(columns={
    'Name': 'name',
    'Measure': 'measure',
    'Geo Place Name': 'geo_place_name',
    'Time Period': 'time_period',
    'Data Value': 'data_value',
    'Year': 'year'
}, inplace=True)


# Asignar un identificador único para cada indicador (esto puede ser generado o asignado en otro proceso)
air_quality_dataset['indicator_id'] = air_quality_dataset.index + 1

# Guardar el nuevo dataset
air_quality_dataset.to_parquet('etl_minio/air_quality_measurement.parquet', index=False)

print("El dataset de calidad del aire ha sido creado y guardado en 'air_quality_measurement.parquet'.")

In [ ]:
air_quality

 Meteorological Data

In [ ]:
import pandas as pd

# Cargar los datos meteorológicos desde el archivo parquet
meteorological_data = pd.read_parquet('Data/datos_meteorologicos_limpios.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
meteorological_data_dataset = meteorological_data[['time', 'temperature_2m (°C)', 'rain (mm)', 'is_day ()']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
meteorological_data_dataset.rename(columns={
    'Time': 'time',
    'Temperature_2m (°C)': 'temperature_2m',
    'rain (mm)': 'rain',
    'is_day': 'is_day'
}, inplace=True)

# Asegurarse de que 'time' sea único (clave primaria)
meteorological_data_dataset.drop_duplicates(subset=['time'], inplace=True)

# Guardar el nuevo dataset
meteorological_data_dataset.to_parquet('etl_minio/meteorological_data.parquet', index=False)

print("El dataset de datos meteorológicos ha sido creado y guardado en 'meteorological_data.parquet'.")


In [ ]:
meteorological_data

Fuel Stations

In [ ]:
import pandas as pd

# Cargar los datos de estaciones de combustible desde el archivo parquet
fuel_stations = pd.read_parquet('Data/df_eafcs.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
fuel_stations_dataset = fuel_stations[['Station Name', 'Street Address', 'City', 'State', 'Fuel Type Code']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
fuel_stations_dataset.rename(columns={
    'Station Name': 'station_name',
    'Street Address': 'street_address',
    'City': 'city',
    'State': 'state',
    'Fuel Type Code': 'fuel_type_code'
}, inplace=True)

# Asegurarse de que 'fuel_type_code' sea único (clave primaria)
fuel_stations_dataset.drop_duplicates(subset=['fuel_type_code'], inplace=True)

# Guardar el nuevo dataset
fuel_stations_dataset.to_parquet('etl_minio/fuel_stations.parquet', index=False)

print("El dataset de estaciones de combustible ha sido creado y guardado en 'fuel_stations.parquet'.")


 Fuel Economy Data

In [2]:
import pandas as pd

# Cargar los datos de economía de combustible desde el archivo parquet
fuel_economy_data = pd.read_parquet('Data/df_vfed.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
fuel_economy_data_dataset = fuel_economy_data[['Manufacturer', 'Model', 'Miles per gallon (mpg)', 'CO2 (p/mile)', 'FuelCost']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
fuel_economy_data_dataset.rename(columns={
    'Manufacturer': 'manufacturer',
    'Model': 'model',
    'Miles per gallon (mpg)': 'miles_per_gallon',
    'CO2 (g p/mile)': 'co2_per_mile',
    'FuelCost': 'fuel_cost'
}, inplace=True)

# Asegurarse de que 'model' sea único (clave primaria)
fuel_economy_data_dataset.drop_duplicates(subset=['model'], inplace=True)

# Guardar el nuevo dataset
fuel_economy_data_dataset.to_parquet('etl_minio/fuel_economy_data.parquet', index=False)

print("El dataset de economía de combustible ha sido creado y guardado en 'fuel_economy_data.parquet'.")


El dataset de economía de combustible ha sido creado y guardado en 'fuel_economy_data.parquet'.


In [ ]:
fuel_economy_data

In [3]:
import pandas as pd
trips = pd.read_parquet('etl_minio/trips.parquet')


In [ ]:
trips

In [3]:
import pandas as pd
import uuid
import pyarrow as pa
import pyarrow.parquet as pq

# Función para generar identificadores únicos
def generate_unique_id():
    return str(uuid.uuid4())

# Cargar datasets
trips = pd.read_parquet('etl_minio/trips.parquet')
vehicles = pd.read_parquet('etl_minio/vehicles.parquet')
geographic_data = pd.read_parquet('etl_minio/geographic_data.parquet')
meteorological_data = pd.read_parquet('etl_minio/meteorological_data.parquet')
fuel_stations = pd.read_parquet('etl_minio/fuel_stations.parquet')
fuel_economy = pd.read_parquet('etl_minio/fuel_economy_data.parquet')
air_quality = pd.read_parquet('etl_minio/air_quality_measurement.parquet')

# Agregar la columna 'unique_column' con identificadores únicos para cada fila
trips['unique_trips'] = [generate_unique_id() for _ in range(len(trips))]
vehicles['unique_vehicles'] = [generate_unique_id() for _ in range(len(vehicles))]
geographic_data['unique_geographic'] = [generate_unique_id() for _ in range(len(geographic_data))]
meteorological_data['unique_meteorological'] = [generate_unique_id() for _ in range(len(meteorological_data))]
fuel_stations['unique_stations'] = [generate_unique_id() for _ in range(len(fuel_stations))]
fuel_economy['unique_economy'] = [generate_unique_id() for _ in range(len(fuel_economy))]
air_quality['unique_air'] = [generate_unique_id() for _ in range(len(air_quality))]

# Guardar los datasets con la nueva columna
# Especificar la compresión (snappy, gzip, brotli, etc.)
compression_type = 'gzip'  # Cambiar por 'gzip', 'brotli', etc.

# Convertir DataFrame a un Table de PyArrow
table = pa.Table.from_pandas(trips)


pq.write_table(table, 'etl_minio/trips.parquet', compression=compression_type)

vehicles.to_parquet('etl_minio/vehicles.parquet', index=False)
geographic_data.to_parquet('etl_minio/geographic_data.parquet', index=False)
meteorological_data.to_parquet('etl_minio/meteorological_data.parquet', index=False)
fuel_stations.to_parquet('etl_minio/fuel_stations.parquet', index=False)
fuel_economy.to_parquet('etl_minio/fuel_economy_data.parquet', index=False)
air_quality.to_parquet('etl_minio/air_quality_measurement.parquet', index=False)


In [2]:
from minio import Minio
from minio.error import S3Error
import os

# Configura tu cliente MinIO
minio_client = Minio(
    "redarcope.ddns.net:9000",  # Reemplaza con tu dirección de MinIO
    access_key="andres",  # Reemplaza con tu access key
    secret_key="andresarcope01",  # Reemplaza con tu secret key
    secure=False  # Cambia a True si usas HTTPS
)


# Lista de archivos a subir
files_to_upload = ["etl_minio/air_quality_measurement.parquet", "etl_minio/fuel_economy_data.parquet","etl_minio/fuel_stations.parquet", "etl_minio/geographic_data.parquet", "etl_minio/meteorological_data.parquet", "etl_minio/trips.parquet", "etl_minio/vehicles.parquet"]
bucket_name = "transport-bucket"

# Verifica si el bucket existe y créalo si no
try:
    if not minio_client.bucket_exists(bucket_name):
        minio_client.make_bucket(bucket_name)
except S3Error as err:
    print("Error al crear el bucket:", err)



for file_path in files_to_upload:
    object_name = os.path.basename(file_path)  # Extrae el nombre del archivo
    try:
        minio_client.fput_object(bucket_name, object_name, file_path)
        print(f"Archivo {object_name} subido con éxito")
    except S3Error as err:
        print(f"Error al subir {object_name}: {err}")


Archivo air_quality_measurement.parquet subido con éxito
Archivo fuel_economy_data.parquet subido con éxito
Archivo fuel_stations.parquet subido con éxito
Archivo geographic_data.parquet subido con éxito
Archivo meteorological_data.parquet subido con éxito
Archivo trips.parquet subido con éxito
Archivo vehicles.parquet subido con éxito
